In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

#check if Metal MPS available
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

tensor([1.], device='mps:0')


In [28]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [29]:
# encode / decode stuff - TOKENIZER
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [30]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(mps_device), y.to(mps_device)
    return x,y

In [31]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(mps_device)

In [35]:
# create a PyTorch optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters+1):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    #sample a batch
    xb, yb = get_batch('train')

    #evaluate loss
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0: train loss 3.1915, val loss 3.1797
step 250: train loss 3.1382, val loss 3.1639
step 500: train loss 3.1319, val loss 3.1582
step 750: train loss 3.0945, val loss 3.1199
step 1000: train loss 3.0905, val loss 3.1105
step 1250: train loss 3.0479, val loss 3.0823
step 1500: train loss 3.0092, val loss 3.0559
step 1750: train loss 3.0208, val loss 3.0445
step 2000: train loss 3.0039, val loss 3.0022
step 2250: train loss 2.9979, val loss 3.0003
step 2500: train loss 2.9524, val loss 2.9922
step 2750: train loss 2.9460, val loss 2.9613
step 3000: train loss 2.9359, val loss 2.9637
step 3250: train loss 2.9210, val loss 2.9459
step 3500: train loss 2.8771, val loss 2.9396
step 3750: train loss 2.8846, val loss 2.9395
step 4000: train loss 2.8630, val loss 2.8983
step 4250: train loss 2.8838, val loss 2.8711
step 4500: train loss 2.8298, val loss 2.8585
step 4750: train loss 2.8392, val loss 2.8554
step 5000: train loss 2.8246, val loss 2.8475
step 5250: train loss 2.7933, val loss 2



has  Heind ad. cand lou
HE45B3W0K&Psiotseenet bWUfirheyf andrd lvwZevZ)Ls w
e!Fe su b tevq)yony aply:;-d tbDj20SxK,_Det'0bud E!3](J'ID t."PqI!T
WANfstof.7go  N_58DITcG0_5))Y!6?erdrJ10y te ar,"CEaw?Iamm sitit";d frkistb-ud e blourof*?lse.
seda an'8limayo d
ld  Zwh4UAow
nd POf aker ane t s ry Zonc-z]s-sx_!98-15VQGX*PWhe,vG
d s got!9are."IZPU3kn'85*0x]4Vts Vctthelin ant r ton thos wicru carm."TJQ&SI'SZness'O]zigeke a terserlBe OWf.C0ZQ, he!j1U!W:(JS5Bfthe s t Gre pyods tht web:ME!silon?-J'szemalig
